# Heat kernel

In [1]:
sigma = 0.0001

## Load packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys
path = '../../scripts/'
sys.path.insert(0,path)
from RipserToDict import ripser_to_dict
from PlotPersistence import plot_persistence
import pickle
import random

## Functions

In [3]:
def heat_kernel(F,G,sigma):
    
    """Let F and G be PH matrices of dimensions n_points x 2, where the columns are birth and death respectively.
    The matrices are thus only for one dim each, e.g. 0 or 1 or 2.
    sigma is a scalar parameter which may be taken to be 0.1.
    The output is the heat kernel"""

    kernel = 0
    for z in G:
        for y in F:
            kernel += np.exp(-np.linalg.norm(y - z,ord=2)**2/(8.*sigma))
            kernel += -np.exp(-np.linalg.norm(y - np.flipud(z),ord=2)**2/(8.*sigma))
    kernel *= 1./(8.*np.pi*sigma)

    return kernel

In [4]:
def heat_kernel_distance(F,G,sigma):
    
    """Let F and G be PH matrices of dimensions n_points x 2, where the columns are birth and death respectively.
    The matrices are thus only for one dim each, e.g. 0 or 1 or 2.
    sigma is a scalar parameter which may be taken to be 0.1.
    The output is the heat kernel distance"""
    
    inner = heat_kernel(F,F,sigma) + heat_kernel(G,G,sigma) - 2*heat_kernel(F,G,sigma)
    distance = np.sqrt(heat_kernel(F,F,sigma) + heat_kernel(G,G,sigma) - 2*heat_kernel(F,G,sigma))
    
    return distance

## Create labels and train kernel

In [5]:
# load train set
train_seeds = range(0,10)
train_persistences = []
train_labels = []
for seed in train_seeds:
    
    # sphere
    with open('../../../heavy_files/exercises/ml_on_1_to_3/sphere_persistences/' + str(seed) + '.txt','rb') as f:
        train_persistences.append(pickle.load(f))
    train_labels.append(0)
    
    # torus
    with open('../../../heavy_files/exercises/ml_on_1_to_3/torus_persistences/' + str(seed) + '.txt','rb') as f:
        train_persistences.append(pickle.load(f))
    train_labels.append(1)

In [6]:
# permute
combined = list(zip(train_persistences, train_labels))
random.shuffle(combined)
train_persistences[:], train_labels[:] = zip(*combined)

In [7]:
train_kernel = np.zeros((len(train_labels),len(train_labels)))
train_homology = [train_persistences[x][1] for x in train_seeds]
for x, i in enumerate(train_homology):
    for y, j in enumerate(train_homology):
        train_kernel[x,y] = heat_kernel(i,j,sigma)
print(train_kernel)

[[  4.07722678e+04   4.72656598e+01   1.43846618e+01   3.11891272e+00
    1.69179557e+00   1.83845475e+04   2.07082065e+04   7.07852195e+01
    2.33727258e+04   2.05062617e+04   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  4.72656598e+01   6.39977740e+04   1.85552780e+04   2.80637097e+04
    2.35689958e+04   1.00341088e+01   4.54561688e+02   2.70155932e+04
    1.20934895e+01   4.88701557e+02   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.43846618e+01   1.85552780e+04   3.61617142e+04   2.13593092e+04
    1.92328638e+04   4.32461049e-07   1.03403923e+02   2.09697121e+04
    1.96786360e+02   5.41424416e-07   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00

In [8]:
# export
with open('train_kernel.txt','wb') as f:
    pickle.dump(train_kernel,f)

with open('train_labels.txt','wb') as f:
    pickle.dump(train_labels,f)

## Create test kernel and test labels

In [9]:
# load test set
test_seeds = range(10,20)
test_persistences = []
test_labels = []
for seed in test_seeds:
    
    # sphere
    with open('../../../heavy_files/exercises/ml_on_1_to_3/sphere_persistences/' + str(seed) + '.txt','rb') as f:
        test_persistences.append(pickle.load(f))
    test_labels.append(0)
    
    # torus
    with open('../../../heavy_files/exercises/ml_on_1_to_3/torus_persistences/' + str(seed) + '.txt','rb') as f:
        test_persistences.append(pickle.load(f))
    test_labels.append(1)

In [10]:
# permute
combined2 = list(zip(test_persistences, test_labels))
random.shuffle(combined2)
test_persistences[:], test_labels[:] = zip(*combined2)

In [11]:
test_kernel = np.zeros((len(test_labels),len(test_labels)))
test_homology = [test_persistences[x][1] for x in range(len(test_seeds))]
for x, i in enumerate(test_homology):
    for y, j in enumerate(test_homology):
        test_kernel[x,y] = heat_kernel(i,j,sigma)
print(test_kernel)

[[  6.71336994e+04   2.61950661e+04   2.18582782e-04   5.46927538e-01
    1.77729476e+02   2.71341708e+04   6.25216669e+01   3.02291416e+04
    1.28563029e+02   3.52999213e+04   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  2.61950661e+04   5.71487361e+04   1.51858293e+02   1.76612398e+01
    1.40268790e+01   2.29290021e+04   4.87428358e-01   2.08872295e+04
    2.32425853e+02   1.98505523e+04   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  2.18582782e-04   1.51858293e+02   3.31300757e+04   2.10398628e+04
    2.00324399e+04   2.05286477e-01   1.81537406e+04   9.32190180e+02
    2.45467798e+04   3.25062114e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00

In [12]:
# export
with open('test_kernel.txt','wb') as f:
    pickle.dump(test_kernel,f)

with open('test_labels.txt','wb') as f:
    pickle.dump(test_labels,f)